In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
from tqdm import tqdm

In [3]:
data_dir = '/content/drive/MyDrive/PLSU/'

In [4]:
data_df = pd.read_excel(data_dir + 'label.xlsx', engine='openpyxl')
data_df

,File Number,life,head,heart,label
0,1,0,0,0,000
1,2,0,0,0,000
2,5,0,1,0,010
3,12,0,0,1,001
4,13,0,1,0,010
...,...,...,...,...,...
1031,5354,1,0,1,101
1032,5366,0,0,0,000
1033,5367,0,0,0,000
1034,5372,0,-,1,0-1


In [5]:
data_df = data_df[(data_df['life'] != '-') & (data_df['life'] != '')]
data_df = data_df[(data_df['head'] != '-') & (data_df['head'] != '')]
data_df = data_df[(data_df['heart'] != '-') & (data_df['heart'] != '')]
data_df

,File Number,life,head,heart,label
0,1,0,0,0,000
1,2,0,0,0,000
2,5,0,1,0,010
3,12,0,0,1,001
4,13,0,1,0,010
...,...,...,...,...,...
1030,5353,0,0,1,001
1031,5354,1,0,1,101
1032,5366,0,0,0,000
1033,5367,0,0,0,000


In [6]:
def reconstruct_file_name(file_number):
    return 'image' + str(file_number) + '.png'

data_df['file'] = data_df['File Number'].apply(reconstruct_file_name)
del data_df['File Number']

data_df

,life,head,heart,label,file
0,0,0,0,000,image1.png
1,0,0,0,000,image2.png
2,0,1,0,010,image5.png
3,0,0,1,001,image12.png
4,0,1,0,010,image13.png
...,...,...,...,...,...
1030,0,0,1,001,image5353.png
1031,1,0,1,101,image5354.png
1032,0,0,0,000,image5366.png
1033,0,0,0,000,image5367.png


In [7]:
mapping_dict = {
    '000': '0',
    '001': '1',
    '010': '2',
    '011': '3',
    '100': '4',
    '101': '5',
    '110': '6',
    '111': '7'
}

data_df['label'] = data_df['label'].apply(lambda x: mapping_dict[x])
data_df

,life,head,heart,label,file
0,0,0,0,0,image1.png
1,0,0,0,0,image2.png
2,0,1,0,2,image5.png
3,0,0,1,1,image12.png
4,0,1,0,2,image13.png
...,...,...,...,...,...
1030,0,0,1,1,image5353.png
1031,1,0,1,5,image5354.png
1032,0,0,0,0,image5366.png
1033,0,0,0,0,image5367.png


In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image

In [9]:
class CustomDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = self.dataframe.iloc[idx, 4]
        image = Image.open(data_dir + 'Mask/' + img_name).convert('RGB')
        label = int(self.dataframe.iloc[idx, 3])

        if self.transform:
            image = self.transform(image)

        return image, label

In [10]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [27]:
from torch.utils.data import random_split

total_samples = len(data_df)
train_size = int(0.8 * total_samples)
valid_size = int(0.1 * total_samples)
test_size = total_samples - train_size - valid_size

dataset = CustomDataset(dataframe=data_df, transform=transform)
train_dataset, valid_dataset, test_dataset = random_split(dataset, [train_size, valid_size, test_size])

# DataLoader for each set
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=8, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=False)

In [28]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()

        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)

        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.downsample = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.downsample = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        residual = self.downsample(x)

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        out += residual
        out = self.relu(out)

        return out

In [29]:
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=8):
        super(ResNet, self).__init__()

        self.in_channels = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(block, 64, layers[0], stride=1)
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)

    def _make_layer(self, block, out_channels, blocks, stride):
        layers = []
        layers.append(block(self.in_channels, out_channels, stride))
        self.in_channels = out_channels
        for _ in range(1, blocks):
            layers.append(block(out_channels, out_channels, stride=1))
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.maxpool(out)

        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)

        out = self.avgpool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)

        return out

In [30]:
# resnet_model = ResNet(ResidualBlock, [2, 2, 2, 2])
resnet_model = ResNet(ResidualBlock, [3, 4, 6, 3])

In [31]:
num_ftrs = resnet_model.fc.in_features
resnet_model.fc = nn.Linear(num_ftrs, 8)

In [32]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet_model.parameters(), lr=0.001)

In [33]:
num_epochs = 30
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet_model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): ResidualBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (downsample): Sequential()
    )
    (1): ResidualBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=Tr

In [35]:
best_valid_accuracy = float('-inf')
best_valid_loss = float('inf')

early_stopping_counter = 0
patience = 3

for epoch in range(num_epochs):
    # Training
    resnet_model.train()
    total_train_loss = 0.0
    correct_train_predictions = 0
    total_train_samples = 0

    for inputs, labels in tqdm(train_dataloader):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = resnet_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_train_loss += loss.item()

        _, predicted = torch.max(outputs, 1)
        correct_train_predictions += (predicted == labels).sum().item()
        total_train_samples += labels.size(0)

    average_train_loss = total_train_loss / len(train_dataloader)
    train_accuracy = correct_train_predictions / total_train_samples
    print(f'Training - Epoch [{epoch+1}/{num_epochs}], Loss: {average_train_loss}, Accuracy: {train_accuracy}')

    # Validation
    resnet_model.eval()
    with torch.no_grad():
        total_valid_loss = 0.0
        correct_valid_predictions = 0
        total_valid_samples = 0

        for inputs, labels in tqdm(valid_dataloader):
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = resnet_model(inputs)
            loss = criterion(outputs, labels)
            total_valid_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            correct_valid_predictions += (predicted == labels).sum().item()
            total_valid_samples += labels.size(0)

        average_valid_loss = total_valid_loss / len(valid_dataloader)
        valid_accuracy = correct_valid_predictions / total_valid_samples
        print(f'Validation - Epoch [{epoch+1}/{num_epochs}], Loss: {average_valid_loss}, Accuracy: {valid_accuracy}')

        # Save the best model
        if valid_accuracy > best_valid_accuracy:
            best_valid_accuracy = valid_accuracy
            torch.save(resnet_model.state_dict(), data_dir + 'best_resnet.pth')
            print('Saved Best Model!!!')

        if average_valid_loss < best_valid_loss:
            best_valid_loss = average_valid_loss
            early_stopping_counter = 0
        else:
            early_stopping_counter += 1

        if early_stopping_counter >= patience:
            print(f'Early stopping.')
            break

    print()

100%|██████████| 25/25 [00:30<00:00,  1.23s/it]


Training - Epoch [1/30], Loss: 1.4392172813415527, Accuracy: 0.38190954773869346


100%|██████████| 13/13 [00:04<00:00,  2.92it/s]


Validation - Epoch [1/30], Loss: 4.623754171224741, Accuracy: 0.13131313131313133
Saved Best Model!!!



100%|██████████| 25/25 [00:31<00:00,  1.26s/it]


...

Validation - Epoch [4/30], Loss: 4.145428694211519, Accuracy: 0.13131313131313133



100%|██████████| 25/25 [00:30<00:00,  1.21s/it]


Training - Epoch [5/30], Loss: 1.2315620851516724, Accuracy: 0.5062814070351759


100%|██████████| 13/13 [00:03<00:00,  3.92it/s]

Validation - Epoch [5/30], Loss: 2.156593451133141, Accuracy: 0.2828282828282828
Early stopping.


In [36]:
# Test
resnet_model.load_state_dict(torch.load(data_dir + 'best_resnet.pth'))
resnet_model.eval()
with torch.no_grad():
    total_test_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    for inputs, labels in tqdm(test_dataloader):
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = resnet_model(inputs)
        loss = criterion(outputs, labels)
        total_test_loss += loss.item()

        _, predicted = torch.max(outputs, 1)
        correct_predictions += (predicted == labels).sum().item()
        total_samples += labels.size(0)

    average_test_loss = total_test_loss / len(test_dataloader)
    accuracy = correct_predictions / total_samples

    print(f'Test - Loss: {average_test_loss}, Accuracy: {accuracy}')

100%|██████████| 13/13 [00:04<00:00,  3.06it/s]

Test - Loss: 2.0243243070749135, Accuracy: 0.3564356435643564
